In [1]:
import numpy as np
import pandas as pd

# pd.set_option('display.height', 1000)
pd.set_option("display.max_rows", 500)
pd.set_option("display.max_columns", 500)
pd.set_option("display.width", 1000)
pd.set_option("max_colwidth", 200)
from IPython.core.display import HTML, display

display(HTML("<style>.container { width:100% !important; }</style>"))


import os

os.environ["R_HOME"] = "/root/miniconda3/envs/R/lib/R"

import rpy2.robjects as objects
from rpy2.robjects.packages import importr

base = importr("base")
r_pROC = importr("pROC")
base._libPaths()[0]


def df_stats(df):
    from tabulate import tabulate

    print("\n***** Shape: ", df.shape, " *****\n")

    columns_list = df.columns.values.tolist()
    isnull_list = df.isnull().sum().values.tolist()
    isunique_list = df.nunique().values.tolist()
    dtypes_list = df.dtypes.tolist()

    list_stat_val = list(zip(columns_list, isnull_list, isunique_list, dtypes_list))
    df_stat_val = pd.DataFrame(
        list_stat_val, columns=["Name", "Null", "Unique", "Dtypes"]
    )
    print(tabulate(df_stat_val, headers="keys", tablefmt="psql"))
    return df.head()

/tmp/ipykernel_108987/253321578.py:9: DeprecationWarning: Importing display from IPython.core.display is deprecated since IPython 7.14, please import from IPython display
  from IPython.core.display import HTML, display


### Convert ECG to PARQUET

In [2]:
df_ecg = pd.read_csv(
    "/media/data1/ravram/DeepECG/ekg_waveforms_output/df_xml_2023_03_14_n_1633856.csv"
)
df_ecg = df_ecg.drop(columns=["Unnamed: 0"])

/tmp/ipykernel_168733/72302712.py:1: DtypeWarning: Columns (2,23,34,108,109,110,176,177,178,180,184,188,200,290,308,315,346,347,351,352,353,354,355,356,357,358,359,360,361,362,363,364,365,366,367,368,372,373,374,381,382,384,385,395,396,397,401,402,403,404,405,406,409,410,413,414,415,416,417,422,423,424,425,426,427,428,429,430,431,432,433,434,436,437) have mixed types. Specify dtype option on import or set low_memory=False.
  df_ecg = pd.read_csv(


In [3]:
df_ecg["RestingECG_PatientDemographics_PatientID"] = df_ecg[
    "RestingECG_PatientDemographics_PatientID"
].astype(str)
df_ecg["RestingECG_TestDemographics_AnalysisSoftwareVersion"] = df_ecg[
    "RestingECG_PatientDemographics_PatientID"
].astype(str)
df_ecg["RestingECG_Order_ExtraADTData1"] = df_ecg[
    "RestingECG_Order_ExtraADTData1"
].astype(str)
df_ecg["RestingECG_ExtraQuestions_ExtraQuestion_0_Answer"] = df_ecg[
    "RestingECG_ExtraQuestions_ExtraQuestion_0_Answer"
].astype(str)

In [4]:
### If column name contains RestingECG_Order  then convert column values to string:
for col in df_ecg.columns:
    if "RestingECG_Order" in col:
        print(col)
        df_ecg[col] = df_ecg[col].astype(str)

In [9]:
### If column name contains RestingECG_Order  then convert column values to string:
for col in df_ecg.columns:
    if "RestingECG_ExtraQuestions_" in col:
        print(col)
        df_ecg[col] = df_ecg[col].astype(str)

RestingECG_ExtraQuestions_ExtraQuestion_0_Question
RestingECG_ExtraQuestions_ExtraQuestion_0_Answer
RestingECG_ExtraQuestions_ExtraQuestion_1_Question
RestingECG_ExtraQuestions_ExtraQuestion_1_Answer
RestingECG_ExtraQuestions_ExtraQuestion_Question
RestingECG_ExtraQuestions_ExtraQuestion_Answer
RestingECG_ExtraQuestions_ExtraQuestion_2_Question
RestingECG_ExtraQuestions_ExtraQuestion_2_Answer
RestingECG_ExtraQuestions_ExtraQuestion_3_Question
RestingECG_ExtraQuestions_ExtraQuestion_3_Answer


In [10]:
# df_ecg.to_parquet(
#    "/media/data1/ravram/DeepECG/ekg_waveforms_output/df_xml_2023_03_14_n_1633856.parquet"
# )

## CHECK ECGs from df_LQTS

In [2]:
df_LQTS = pd.read_csv("data/LQTS/DeepECG-LQT-20230210.csv", delim_whitespace=True)
display(df_LQTS.head())

,Dossier,Genotype
0,101848,Negative
1,118531,Negative
2,133506,Negative
3,200443,Negative
4,207024,LQT2


In [3]:
df_ecg = pd.read_parquet(
    "/media/data1/ravram/DeepECG/ekg_waveforms_output/df_xml_2023_03_14_n_1633856.parquet"
)

In [4]:
# Drop rows where RestingECG_PatientDemographics_PatientID is not an int
df_ecg = df_ecg[df_ecg["RestingECG_PatientDemographics_PatientID"].str.isdigit()]
df_ecg["RestingECG_PatientDemographics_PatientID"] = pd.to_numeric(
    df_ecg["RestingECG_PatientDemographics_PatientID"], errors="coerce"
)
df_ecg["RestingECG_PatientDemographics_PatientID"] = (
    df_ecg["RestingECG_PatientDemographics_PatientID"].fillna(0).astype(int)
)

##df_ecg = df_ecg['RestingECG_PatientDemographics_PatientID'].astype(int, errors='coerce')

In [5]:
## Check with of df_LQTS['Dossier'] is in df_ecg['RestingECG_PatientDemographics_PatientID']
df_LQTS["Dossier"] = df_LQTS["Dossier"].astype(int)
## Check if df_LQTS['Dossier'] is in df_ecg['RestingECG_PatientDemographics_PatientID']
df_LQTS["Dossier"] = df_LQTS["Dossier"].astype(int)
df_ecg["RestingECG_PatientDemographics_PatientID"] = df_ecg[
    "RestingECG_PatientDemographics_PatientID"
].astype(int)

In [6]:
df_LQTS["Dossier_exists_in_ecg"] = df_LQTS["Dossier"].isin(
    df_ecg["RestingECG_PatientDemographics_PatientID"]
)
display(df_LQTS["Dossier_exists_in_ecg"].value_counts())

True     187
False     12
Name: Dossier_exists_in_ecg, dtype: int64

In [7]:
df_ecg["Dossier_exists_in_ecg"] = df_ecg[
    "RestingECG_PatientDemographics_PatientID"
].isin(df_LQTS["Dossier"])

In [8]:
## Check if file at df_ecg['xml_path'] exists if df_ecg['Dossier_exists_in_ecg'] == True
# df_ecg = df_ecg.loc[df_ecg['Dossier_exists_in_ecg']==True]
# df_ecg['xml_path_exists'] = df_ecg['xml_path'].apply(lambda x: os.path.exists(x))
# df_ecg['xml_path_exists'] = df_ecg['xml_path_exists'].astype(int)

In [24]:
#
# df_ecg.loc[df_ecg['Dossier_exists_in_ecg']==True].to_csv('../lqts/data/ecg_lqts_rafik.csv')
# display(df_ecg['xml_path_exists'].value_counts())

df_ecg = pd.read_csv("../lqts/data/df_lqts_merged.csv")
display(df_ecg.head(n=2))

/tmp/ipykernel_108987/819029784.py:5: DtypeWarning: Columns (19,35,110,111,178,179,181,189,347,353,365,411,433) have mixed types. Specify dtype option on import or set low_memory=False.
  df_ecg = pd.read_csv('../lqts/data/df_lqts_merged.csv')


,Unnamed: 0.1,Unnamed: 0,RestingECG_MuseInfo_MuseVersion,patient_id,age,RestingECG_PatientDemographics_AgeUnits,dob,sex,RestingECG_PatientDemographics_PatientLastName,RestingECG_PatientDemographics_PatientFirstName,RestingECG_TestDemographics_DataType,RestingECG_TestDemographics_Site,RestingECG_TestDemographics_SiteName,RestingECG_TestDemographics_AcquisitionDevice,RestingECG_TestDemographics_Status,RestingECG_TestDemographics_EditListStatus,RestingECG_TestDemographics_Priority,RestingECG_TestDemographics_Location,RestingECG_TestDemographics_LocationName,id_site,RestingECG_TestDemographics_AcquisitionTime,RestingECG_TestDemographics_AcquisitionDate,RestingECG_TestDemographics_CartNumber,RestingECG_TestDemographics_AcquisitionSoftwareVersion,RestingECG_TestDemographics_AnalysisSoftwareVersion,RestingECG_TestDemographics_EditTime,RestingECG_TestDemographics_EditDate,RestingECG_TestDemographics_AcquisitionTechID,RestingECG_TestDemographics_EditorID,RestingECG_TestDemographics_TestReason,RestingECG_TestDemographics_AcquisitionTechLastName,RestingECG_TestDemographics_AcquisitionTechFirstName,RestingECG_TestDemographics_EditorLastName,RestingECG_TestDemographics_EditorFirstName,RestingECG_TestDemographics_HISStatus,RestingECG_Order_ExtraADTData1,RestingECG_RestingECGMeasurements_VentricularRate,RestingECG_RestingECGMeasurements_AtrialRate,RestingECG_RestingECGMeasurements_PRInterval,RestingECG_RestingECGMeasurements_QRSDuration,RestingECG_RestingECGMeasurements_QTInterval,qt_manual,RestingECG_RestingECGMeasurements_PAxis,RestingECG_RestingECGMeasurements_RAxis,RestingECG_RestingECGMeasurements_TAxis,RestingECG_RestingECGMeasurements_QRSCount,RestingECG_RestingECGMeasurements_QOnset,RestingECG_RestingECGMeasurements_QOffset,RestingECG_RestingECGMeasurements_POnset,RestingECG_RestingECGMeasurements_POffset,RestingECG_RestingECGMeasurements_TOffset,RestingECG_RestingECGMeasurements_ECGSampleBase,RestingECG_RestingECGMeasurements_ECGSampleExponent,RestingECG_RestingECGMeasurements_QTcFrederica,RestingECG_OriginalRestingECGMeasurements_VentricularRate,RestingECG_OriginalRestingECGMeasurements_AtrialRate,RestingECG_OriginalRestingECGMeasurements_PRInterval,RestingECG_OriginalRestingECGMeasurements_QRSDuration,RestingECG_OriginalRestingECGMeasurements_QTInterval,qt_manual_confirmed,RestingECG_OriginalRestingECGMeasurements_PAxis,RestingECG_OriginalRestingECGMeasurements_RAxis,RestingECG_OriginalRestingECGMeasurements_TAxis,RestingECG_OriginalRestingECGMeasurements_QRSCount,RestingECG_OriginalRestingECGMeasurements_QOnset,RestingECG_OriginalRestingECGMeasurements_QOffset,RestingECG_OriginalRestingECGMeasurements_POnset,RestingECG_OriginalRestingECGMeasurements_POffset,RestingECG_OriginalRestingECGMeasurements_TOffset,RestingECG_OriginalRestingECGMeasurements_ECGSampleBase,RestingECG_OriginalRestingECGMeasurements_ECGSampleExponent,RestingECG_OriginalRestingECGMeasurements_QTcFrederica,RestingECG_Diagnosis_Modality,RestingECG_Diagnosis_DiagnosisStatement_0_StmtFlag,RestingECG_Diagnosis_DiagnosisStatement_0_StmtText,RestingECG_Diagnosis_DiagnosisStatement_1_StmtFlag,RestingECG_Diagnosis_DiagnosisStatement_1_StmtText,RestingECG_Diagnosis_DiagnosisStatement_2_StmtText,RestingECG_Diagnosis_DiagnosisStatement_3_StmtText,RestingECG_Diagnosis_DiagnosisStatement_4_StmtText,RestingECG_Diagnosis_DiagnosisStatement_5_StmtFlag,RestingECG_Diagnosis_DiagnosisStatement_5_StmtText,RestingECG_Diagnosis_DiagnosisStatement_6_StmtText,RestingECG_Diagnosis_DiagnosisStatement_7_StmtFlag,RestingECG_Diagnosis_DiagnosisStatement_7_StmtText,RestingECG_Diagnosis_DiagnosisStatement_8_StmtText,RestingECG_Diagnosis_DiagnosisStatement_9_StmtFlag,RestingECG_Diagnosis_DiagnosisStatement_9_StmtText,RestingECG_Diagnosis_DiagnosisStatement_10_StmtFlag,RestingECG_Diagnosis_DiagnosisStatement_10_StmtText,RestingECG_OriginalDiagnosis_Modality,RestingECG_OriginalDiagnosis_DiagnosisStatement_0_StmtFlag,RestingECG_OriginalDiagnosis_DiagnosisStatement_0_StmtText,RestingECG_OriginalDi

In [29]:
## Write a function to copy ECGs from df_ecg['xml_path'] to local folder and make 'folder' if it doesn't exist


def copy_ecg_to_local_folder(df_ecg, output_folder="lqts_ecg/"):
    import shutil

    from tqdm import tqdm

    # If output_folder doesn't exist, create it
    if not os.path.exists(output_folder):
        os.makedirs(output_folder)

    for index, row in tqdm(df_ecg.iterrows()):
        if row["Dossier_exists_in_ecg"] == True:
            print(row["xml_path"])
            print(row["lqts_type"])
            ## Copy row['xml_path'] to output_folder
            shutil.copy(row["xml_path"], output_folder)

In [30]:
copy_ecg_to_local_folder(df_ecg[0:10])

10it [00:00, 542.88it/s]

/media/data1/muse_ge/ecg_retrospective/MUSE_20220816_102934_05000.xml
Control
/media/data1/muse_ge/ecg_retrospective/MUSE_20220816_114548_97000.xml
Control
/media/data1/muse_ge/ecg_retrospective/MUSE_20220816_120634_22000.xml
Control
/media/data1/muse_ge/ecg_retrospective/MUSE_20220816_135832_30000.xml
Control
/media/data1/muse_ge/ecg_retrospective/MUSE_20220816_144313_26000.xml
Control
/media/data1/muse_ge/ecg_retrospective/MUSE_20220816_144831_33000.xml
Type 2
/media/data1/muse_ge/ecg_retrospective/MUSE_20220816_144833_78000.xml
Type 2
/media/data1/muse_ge/ecg_retrospective/MUSE_20220816_162325_42000.xml
Control
/media/data1/muse_ge/ecg_retrospective/MUSE_20220816_164253_93000.xml
Control
/media/data1/muse_ge/ecg_retrospective/MUSE_20220816_164255_23000.xml
Control


In [31]:
import os
import shutil

import matplotlib.pyplot as plt
## Load the first df_ecg['npy_path'] and detemrine the shape
import numpy as np
import pandas as pd
from tqdm import tqdm

ecg_array = np.load(df_ecg["npy_path"].iloc[0])

In [32]:
display(ecg_array.shape)

(2500, 12, 1)